## Thomson Numerical Optimization

In [ ]:
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Parameters
N = 3  # Number of points
lr = 0.01  # Learning rate
num_steps = 1000  # Number of optimization steps

# Initialize points randomly on the sphere
points = torch.randn(N, 3)  # Create without gradients
points /= points.norm(dim=1, keepdim=True)  # Normalize to lie on the sphere
points.requires_grad_()  # Enable gradients

# Optimizer
optimizer = torch.optim.Adam([points], lr=lr)

# Optimization loop
for step in range(num_steps):
    # Zero the gradients
    optimizer.zero_grad()

    # Compute the energy
    energy = 0
    for i in range(N):
        for j in range(i + 1, N):
            distance = torch.norm(points[i] - points[j])
            energy += 1 / distance  # Coulomb potential

    # Backpropagate to compute gradients
    energy.backward()

    # Project gradients onto the tangent space of the sphere
    with torch.no_grad():
        grad = points.grad
        tangent_grad = grad - (grad * points).sum(dim=1, keepdim=True) * points
        points.grad.copy_(tangent_grad)  # Replace the gradient with the tangent component

    # Take an optimizer step
    optimizer.step()

    # Reproject points back onto the sphere
    with torch.no_grad():
        points.data /= points.data.norm(dim=1, keepdim=True)

    # Visualization every 10 steps
    if step % 10 == 0 or step == steps - 1:
        print(f"Step {step}, Energy: {energy.item()}")
        clear_output(wait=True)  # Clear previous plot
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection='3d')
        plot_thomson(ax, points)
        plt.title(f"Step {step}, Energy: {energy.item():.4f}")
        plt.show()

# Final result
print("Optimized Points:")
print(points)
